<a href="https://colab.research.google.com/github/smfan-1/quality-check/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing libraries
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import serial
import time

In [ ]:
# Dataset path
data_dir = "/content/data"

In [ ]:
# Image parameters
img_height, img_width = 128, 128
batch_size = 15

In [ ]:
# Data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [ ]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 232 images belonging to 3 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 58 images belonging to 3 classes.


In [ ]:
# CNN Model
def build_model():
    model = keras.Sequential([
        layers.Conv2D(15, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(60, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(180, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(180, activation='relu'),
        layers.Dropout(0.5),
        # Note: Adjust the number of output classes as needed.
        layers.Dense(3, activation='softmax')  # 5 categories (defects + original)
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 126, 126, 15)   │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 63, 63, 15)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 61, 61, 60)     │         8,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 30, 30, 60)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 28, 28, 180)    │        97,380 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 14, 14, 180)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 35280)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 180)            │     6,350,580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 180)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 3)              │           543 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,457,083 (24.63 MB)

 Trainable params: 6,457,083 (24.63 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train model
epochs = 15
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step - accuracy: 0.5194 - loss: 0.8406

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 333ms/step - accuracy: 0.5294 - loss: 0.8279 - val_accuracy: 0.5345 - val_loss: 1.0697
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 265ms/step - accuracy: 0.8867 - loss: 0.2763 - val_accuracy: 0.5172 - val_loss: 2.1692
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 299ms/step - accuracy: 0.9576 - loss: 0.1249 - val_accuracy: 0.5345 - val_loss: 2.2015
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 263ms/step - accuracy: 0.9857 - loss: 0.0738 - val_accuracy: 0.6724 - val_loss: 3.3545
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 280ms/step - accuracy: 0.9894 - loss: 0.0301 - val_accuracy: 0.5690 - val_loss: 3.1103
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 260ms/step - accuracy: 0.9860 - loss: 0.0228 - val_accuracy: 0.6724 - val_loss: 2.9223
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 263ms/step - accuracy: 0.9965 - loss: 0.0162 - val_accuracy: 0.7069 - val_loss: 2.6036
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 305ms/step - accuracy: 1.0000 - loss: 0.0062 - val_accuracy: 0.7414 - val_

In [ ]:
# Save model
model.save("product_quality.h5")